# Intrusion Detection with Logistic Regression
In questo notebook utilizzeremo Logistic Regression per classificare i flussi di traffico di rete come benigni o dannosi. Il modello di Logistic Regression restituisce un valore compreso tra 0 e 1, che rappresenta la probabilità che il flusso in ingresso sia dannoso. Usiamo una soglia fissata a 0,5 per determinare se il flusso di rete è dannoso o meno.
Addestreremo un modello di Logistic Regression con traffico di rete benigno e quattro classi di attacchi DDoS dal dataset CIC-DDoS2019 dell’Università del New Brunswick. Il traffico di rete è stato precedentemente pre-elaborato in modo che i pacchetti siano raggruppati in flussi di traffico bidirezionali utilizzando la 5-tupla (IP sorgente, IP destinazione, porta sorgente, porta destinazione, protocollo). Ogni flusso è rappresentato da 21 features (attributi) dell’header dei pacchetti calcolate da un massimo di 1000 pacchetti:

| Features           | Logistic Regression model           |
|---------------------|--------------------|
| timestamp (mean IAT)  <br> packet_length (mean) <br> IP_flags_df (sum) <br> IP_flags_mf (sum) <br> IP_flags_rb (sum) <br> IP_frag_off (sum) <br> protocols (mean) <br> TCP_length (mean) <br> TCP_flags_ack (sum) <br> TCP_flags_cwr (sum) <br> TCP_flags_ece (sum) <br> TCP_flags_fin (sum) <br> TCP_flags_push (sum) <br> TCP_flags_res (sum) <br> TCP_flags_reset (sum) <br> TCP_flags_syn (sum) <br> TCP_flags_urg (sum) <br> TCP_window_size (mean) <br> UDP_length (mean) <br> ICMP_type (mean) <br> Packets (counter) <br>| <img src="./logistic_regression_CIC2019.png" width="100%">  |

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Corso di Algoritmi di Machine Learning per la rilevazione di attacchi informatici
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import necessary libraries

import numpy as np
import glob
import h5py
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD,Adam
from util_functions import *
DATASET_FOLDER = "../Dataset"

# Conversione delle labels
Nel nostro dataset, le labels assegnate ai flussi hanno un valore numerico che varia da 0 (benigno) a 4 (abbiamo 4 tipi di attacchi). In questo notebook, noi ci occupiamo di classificazione binaria (0/1, cioe' benigno/malevolo) e per cui, dopo aver caricato il dataset dai files, convertiamo tutte le labels > 0 in 1. 

Questo vuol dire che non ci interessa capire il tipo di attacco, ma solo se c'e' un attacco o meno.

In [ ]:
# Load training, validation and test sets
feature_names = get_feature_names()
target_names = ['benign', 'dns',  'syn', 'udplag', 'webddos'] 
target_names_full = ['benign', 'dns', 'ldap', 'mssql', 'netbios', 'ntp', 'portmap', 'snmp', 'ssdp', 'syn', 'tftp', 'udp', 'udplag', 'webddos'] 
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')
y_train = np.array([1 if y > 0 else 0 for y in y_train]) # classificazione binaria, tutte le labels diverse da 0 le trasformiamo in 1

X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5')
y_val = np.array([1 if y > 0 else 0 for y in y_val]) # classificazione binaria, tutte le labels diverse da 0 le trasformiamo in 1

X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')
y_test = np.array([1 if y > 0 else 0 for y in y_test]) # classificazione binaria, tutte le labels diverse da 0 le trasformiamo in 1

# Implementazione del modello
Nella prossima cella, implementiamo il modello di Logistic Regression. 
Il modello e' creato con una procedura identica a quella usata per creare una rete neurale. L'unica differeza sta nella complessita' del modello. Nel caso di Logistic Regression basta una riga di codice (riga 4). Nel caso di una rete neurale profonda, no. 

In [ ]:
# Logistic Regression model
def LogRegression(model_name, input_shape):
    model = Sequential(name=model_name)
    model.add(Dense(1, input_shape=input_shape,activation='sigmoid', name='fc1'))

    print(model.summary())
    return model

def compileModel(model,lr):
    #optimizer = SGD(learning_rate=lr, momentum=0.0) # the optimisation algorithm
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])  # here we specify the loss function

# Model training
Nella cella seguente addestriamo il modello di Logistic Regression. L'output del processo di training mostra come l'accuracy del modello tenda a crescere ad ogni iterazione (```epoch```). Questo vuol dire che il modello sta imparando a distiguere i flussi di traffico benigni da quelli malevoli.
Lo addestriamo per 100 epoche. E' sufficiente?

In [ ]:
model = LogRegression('log_reg', X_train.shape[1:4])
compileModel(model,0.001)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_val, y_val))

# Usiamo il modello con dei dati mai visti (test set)

In [ ]:
y_pred = np.squeeze(model.predict(X_test, batch_size=32) > 0.5)

print("F1 Score: ", f1_score(y_test,y_pred))

# Stampiamo la confusion matrix
Questo ci permette di capire dove sbaglia il modello.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
np.set_printoptions(precision=2)

y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)

cm = confusion_matrix(y_test_binary, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Benigno', 'DDoS'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

# Early stopping
Invece di addestrare per un numero pre-definito di epoche, usiamo la tecnica chiamata ```early stopping```. Early stopping controlla l'accuratezza del modello ad ogni epoca. Se l'accuratezza non cresce per un certo numero di epoche (```PATIENCE```), il processo di training si ferma.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = LogRegression('log_reg', X_train.shape[1:4])
compileModel(model,0.001)

PATIENCE = 25 

early_stopping = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, verbose=1, restore_best_weights=True)

# Retrain the model with early-stopping
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])

# Stampiamo di nuovo la confusion matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
np.set_printoptions(precision=2)

y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)

cm = confusion_matrix(y_test_binary, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Benigno', 'DDoS'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()